<a href="https://colab.research.google.com/github/NitinGautam05/Plant-leaf-disease-identification/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import requests
import zipfile
import io

# URL of the ZIP file
url = "https://data.mendeley.com/public-files/datasets/tywbtsjrjv/files/d5652a28-c1d8-4b76-97f3-72fb80f94efc/file_downloaded"

# Send an HTTP GET request to the URL
response = requests.get(url)

# Check if the request was successful (status code 200)
if response.status_code == 200:
    # Open the ZIP file
    with zipfile.ZipFile(io.BytesIO(response.content), "r") as zip_ref:
        # Extract all the contents to a directory
        zip_ref.extractall("")
    print("Extraction successful.")
else:
    print("Failed to download the ZIP file.")

Extraction successful.


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator

data_dir = 'Plant_leave_diseases_dataset_without_augmentation'

# Define image dimensions and batch size
img_width, img_height = 256, 256
batch_size = 32

# Use ImageDataGenerator to load and preprocess images
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation'
)

test_generator = train_datagen.flow_from_directory(
    data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

print("Number of training samples:", train_generator.samples)
print("Number of validation samples:", validation_generator.samples)
print("Number of test samples:", test_generator.samples)


Found 44371 images belonging to 39 classes.
Found 11077 images belonging to 39 classes.
Found 55448 images belonging to 39 classes.
Number of training samples: 44371
Number of validation samples: 11077
Number of test samples: 55448


In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16

# Load the pre-trained VGG16 model
vgg_base = VGG16(weights='imagenet', include_top=False, input_shape=(img_width, img_height, 3))

# Freeze the convolutional base
vgg_base.trainable = False


58889256/58889256 [==============================] - 0s 0us/step


In [ ]:

# Add custom classification layers on top
model = models.Sequential([
    vgg_base,
    layers.Flatten(),
    layers.Dense(256, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(train_generator.num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])



In [ ]:
# Train the model
history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // batch_size
)


Epoch 1/10
1386/1386 [==============================] - 316s 221ms/step - loss: 1.5706 - accuracy: 0.5273 - val_loss: 0.5669 - val_accuracy: 0.8298
Epoch 2/10
1386/1386 [==============================] - 282s 203ms/step - loss: 1.0866 - accuracy: 0.6347 - val_loss: 0.4233 - val_accuracy: 0.8671
Epoch 3/10
1386/1386 [==============================] - 281s 203ms/step - loss: 0.9777 - accuracy: 0.6631 - val_loss: 0.4463 - val_accuracy: 0.8531
Epoch 4/10
1386/1386 [==============================] - 304s 219ms/step - loss: 0.9206 - accuracy: 0.6782 - val_loss: 0.3659 - val_accuracy: 0.8945
Epoch 5/10
1386/1386 [==============================] - 304s 219ms/step - loss: 0.8808 - accuracy: 0.6918 - val_loss: 0.3576 - val_accuracy: 0.8861
Epoch 6/10
1386/1386 [==============================] - 279s 201ms/step - loss: 0.8855 - accuracy: 0.6925 - val_loss: 0.3577 - val_accuracy: 0.8894
Epoch 7/10
1386/1386 [==============================] - 303s 219ms/step - loss: 0.8408 - accuracy: 0.7051 - val_

In [ ]:

# Evaluate the model on the test set
test_loss, test_acc = model.evaluate(test_generator, steps=test_generator.samples // batch_size)
print("Test accuracy:", test_acc)

# Predict labels for the test set
test_pred = model.predict(test_generator, steps=test_generator.samples // batch_size)
test_labels = np.argmax(test_generator.labels, axis=1)
test_pred_labels = np.argmax(test_pred, axis=1)

# Calculate precision, recall, and F1-score
print(classification_report(test_labels, test_pred_labels))


1732/1732 [==============================] - 274s 158ms/step - loss: 0.2113 - accuracy: 0.9359
Test accuracy: 0.9358761310577393
